# --- Day 17: Spinlock ---

Suddenly, whirling in the distance, you notice what looks like a massive, pixelated hurricane: a deadly spinlock. This spinlock isn't just consuming computing power, but memory, too; vast, digital mountains are being ripped from the ground and consumed by the vortex.

If you don't move quickly, fixing that printer will be the least of your problems.

This spinlock's algorithm is simple but efficient, quickly consuming everything in its path. It starts with a circular buffer containing only the value 0, which it marks as the current position. It then steps forward through the circular buffer some number of steps (your puzzle input) before inserting the first new value, 1, after the value it stopped on. The inserted value becomes the current position. Then, it steps forward from there the same number of steps, and wherever it stops, inserts after it the second new value, 2, and uses that as the new current position again.

It repeats this process of stepping forward, inserting a new value, and using the location of the inserted value as the new current position a total of `2017` times, inserting 2017 as its final operation, and ending with a total of 2018 values (including 0) in the circular buffer.

For example, if the spinlock were to step 3 times per insert, the circular buffer would begin to evolve like this (using parentheses to mark the current position after each iteration of the algorithm):

- `(0)`, the initial state before any insertions.
- `0 (1)`: the spinlock steps forward three times (0, 0, 0), and then inserts the first value, 1, after it. 1 becomes the current position.
- `0 (2) 1`: the spinlock steps forward three times (0, 1, 0), and then inserts the second value, 2, after it. 2 becomes the current position.
- `0  2 (3) 1`: the spinlock steps forward three times (1, 0, 2), and then inserts the third value, 3, after it. 3 becomes the current position.

And so on:

```
0  2 (4) 3  1
0 (5) 2  4  3  1
0  5  2  4  3 (6) 1
0  5 (7) 2  4  3  6  1
0  5  7  2  4  3 (8) 6  1
0 (9) 5  7  2  4  3  8  6  1
```

Eventually, after 2017 insertions, the section of the circular buffer near the last insertion looks like this:

1512  1134  151 (2017) 638  1513  851
Perhaps, if you can identify the value that will ultimately be after the last value written (2017), you can short-circuit the spinlock. In this example, that would be 638.

**What is the value after 2017 in your completed circular buffer?**

For part 1, using a regular list or a deque with inserts works just fine:

In [189]:
puzzle_input = 335

In [190]:
from collections import deque

def circular_buffer(steps=3, cycles=2017, verbose=False):
    buff = deque([0])
    pos = 0
    
    for i in range(1, cycles+1):
        new_pos = 1 + (pos + steps) % len(buff)
        buff.insert(new_pos, i)
        pos = new_pos
        if verbose:
            print(buff)
    print("i", i)
    #print(buff[pos-1:pos+2])
    return buff[new_pos+1]

circular_buffer(3,20, True)
circular_buffer(3, 2017)

deque([0, 1])
deque([0, 2, 1])
deque([0, 2, 3, 1])
deque([0, 2, 4, 3, 1])
deque([0, 5, 2, 4, 3, 1])
deque([0, 5, 2, 4, 3, 6, 1])
deque([0, 5, 7, 2, 4, 3, 6, 1])
deque([0, 5, 7, 2, 4, 3, 8, 6, 1])
deque([0, 9, 5, 7, 2, 4, 3, 8, 6, 1])
deque([0, 9, 5, 7, 2, 10, 4, 3, 8, 6, 1])
deque([0, 9, 5, 7, 2, 10, 4, 3, 8, 11, 6, 1])
deque([0, 12, 9, 5, 7, 2, 10, 4, 3, 8, 11, 6, 1])
deque([0, 12, 9, 5, 7, 13, 2, 10, 4, 3, 8, 11, 6, 1])
deque([0, 12, 9, 5, 7, 13, 2, 10, 4, 14, 3, 8, 11, 6, 1])
deque([0, 12, 9, 5, 7, 13, 2, 10, 4, 14, 3, 8, 11, 15, 6, 1])
deque([0, 16, 12, 9, 5, 7, 13, 2, 10, 4, 14, 3, 8, 11, 15, 6, 1])
deque([0, 16, 12, 9, 5, 17, 7, 13, 2, 10, 4, 14, 3, 8, 11, 15, 6, 1])
deque([0, 16, 12, 9, 5, 17, 7, 13, 2, 18, 10, 4, 14, 3, 8, 11, 15, 6, 1])
deque([0, 16, 12, 9, 5, 17, 7, 13, 2, 18, 10, 4, 14, 19, 3, 8, 11, 15, 6, 1])
deque([0, 16, 12, 9, 5, 17, 7, 13, 2, 18, 10, 4, 14, 19, 3, 8, 11, 20, 15, 6, 1])
i 20
i 2017


638

So that works for the test case, now to try it out on the puzzle input:

In [191]:
circular_buffer(puzzle_input, 2017)

i 2017


1282

# --- Part Two ---

The spinlock does not short-circuit. Instead, it gets more angry. At least, you assume that's what happened; it's spinning significantly faster than it was a moment ago.

You have good news and bad news.

The good news is that you have improved calculations for how to stop the spinlock. They indicate that you actually need to identify the value after 0 in the current state of the circular buffer.

The bad news is that while you were determining this, the spinlock has just finished inserting its fifty millionth value (50000000).

**What is the value after 0 the moment 50000000 is inserted?**

Turns out my implementation above is very slow, so I need to rewrite the function to speed it up. the internet suggests deque, or numpy. The main slowdown seems to be inserting in the middle of a python list, so I need a solution which appends at the end:

Using either a linked list or deque with inserts in the middle is very slow, so need to rewrite:

In [192]:
from collections import deque

circular_buffer(puzzle_input, 5*10**7)

KeyboardInterrupt: 

## part 2 solution without using a list

A different approach: zero is always the first number in the circular buffer, as if the position is at the end the list gets one bigger, and if at the start the number is inserted after the first number, which is zero and never changes.

Thus we only have to track what happens to the number after zero:

In [184]:
def circular_buffer_zero(steps=puzzle_input, cycles=5*10**7):
    buff = deque([0])
    pos = 0
    
    num_after_zero = 0
    
    for i in range(1, cycles+1):
        new_pos = (pos + steps) % i
        if new_pos == 0:
            num_after_zero = i
        pos = new_pos + 1
        #print(buff)
    
    print("i", i)
    return num_after_zero

circular_buffer_zero()

i 50000000


27650600